# Interakcija SSH/Telnet

## SSH

### Installation

    pip3 install paramiko scp

### Making a connection

In [ ]:
import paramiko

ssh_client=paramiko.SSHClient()
ssh_client.connect(hostname='212.101.137.47', port=22, username='sshtest', password='')
#paramiko.ssh_exception.SSHException: Server '[192.168.56.101]:2022' not found in known_hosts 

In [ ]:
import paramiko
ssh_client =paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# bolj varna opcija - https://docs.paramiko.org/en/stable/api/client.html#paramiko.client.SSHClient.get_host_keys
#ssh_client.load_system_host_keys()
ssh_client.connect(hostname='212.101.137.47', port=22, username='sshtest', password='')
# neki delamo 
print('Working')
ssh_client.close()

#### Setting up SSH Keys

    ssh-keygen -t rsa

    ssh-copy-id -i ~/.ssh/id_rsa.pub testuser@ssh-telnet-test

In [ ]:
import paramiko
ssh_client =paramiko.SSHClient()
#ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# bolj varna opcija - https://docs.paramiko.org/en/stable/api/client.html#paramiko.client.SSHClient.get_host_keys
ssh_client.load_system_host_keys()
ssh_client.connect(hostname='212.101.137.47', port=22, username='sshtest', passphrase='leons')
# neki delamo 
print('Working')
ssh_client.close()

### Running commands on the remote machine

    stdin,stdout,stderr=ssh_client.exec_command(“ls”)

    # branje vrednosti
    print(stdout.readlines())
    print(stderr.readlines())

In [ ]:
import paramiko

def execute_command(client, command):
    """Execute command in succession."""
    stdin, stdout, stderr = client.exec_command(command)
    status = stdout.channel.recv_exit_status()
    if status != 0:
        print(f'Status: {status}, message: {stderr.readlines()}')
    response = stdout.readlines()
    client.close()
    print(response)

ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_client.connect(hostname='ssh-telnet-test', port=22, username='testuser', password='test')
command = "uname -a"

execute_command(ssh_client, command)

#### COMMANDS REQUIRING INPUT

In [ ]:
ssh_client.load_system_host_keys()
ssh_client.connect(hostname='212.101.137.47', port=22, username='sshtest', passphrase='leons')

stdin, stdout, stderr = ssh_client.exec_command("sudo ls -la")
stdin.write("EnoVelikoDrevoCesenj12!")
stdin.write("\n")
stdin.flush()
print(stdout.readlines())
print(stderr.readlines())

print('Working')
ssh_client.close()

## Primer: Create a SSH/SCP program

### Starting our Script

In [ ]:
# config.py
"""Configuration module."""
from os import environ

host = environ.get('REMOTE_HOST')
user = environ.get('REMOTE_USERNAME')
password = environ.get('REMOTE_PASSWORD')

remote_path = environ.get('REMOTE_PATH')
local_path = '/home/plume/data'

### Creating an SSH Client

In [ ]:
# client.py
class RemoteClient:
    """Client to interact with a remote host via SSH & SCP."""

    def __init__(self, host, user, password, remote_path):
        self.host = host
        self.user = user
        self.password = password
        self.remote_path = remote_path
        self.client = None
        self.scp = None

In [ ]:
# client.py
"""Remote host object to handle connections and actions."""
from paramiko import SSHClient, AutoAddPolicy, RSAKey
from paramiko.auth_handler import AuthenticationException, SSHException


class RemoteClient:
    """Client to interact with a remote host via SSH & SCP."""

    def __init__(self, host, user, password, remote_path):
        self.host = host
        self.user = user
        self.password = password
        self.remote_path = remote_path
        self.client = None
        self.scp = None


### Uploading SSH Keys to a Remote Host

In [ ]:
"""Remote host object to handle connections and actions."""
import sys
from loguru import logger
from os import system
from paramiko import SSHClient, AutoAddPolicy
from scp import SCPClient, SCPException


logger.add(sys.stderr,
           format="{time} {message}",
           filter="client",
           level="INFO")
logger.add('logs/log_{time:YYYY-MM-DD}.log',
           format="{time} {level} {message}",
           filter="client",
           level="ERROR")


class RemoteClient:
    """Client to interact with a remote host via SSH & SCP."""
    def __init__(self, host, user, password=None, passphrase=None, remote_path='~/', port=22):
        self.host = host
        self.user = user
        self.password = password
        self.passphrase = passphrase
        self.remote_path = remote_path
        self.port = port
        self.client = None
        self.scp = None

    RSAKey.from_private_key_file(self.ssh_key_filepath)

    ssh-copy-id -i ~/.ssh/mykey user@host

    system(f'ssh-copy-id -i {self.ssh_key_filepath} {self.user}@{self.host}>/dev/null 2>&1')

> \> is for redirect /dev/null is a black hole where any data sent, will be discarded. 2 is the file descriptor for Standard Error. > is for redirect. & is the symbol for file descriptor (without it, the following 1 would be considered a filename). 1 is the file descriptor for Standard O.

### Connecting to our Client

In [ ]:
class RemoteClient:
    """Client to interact with a remote host via SSH & SCP."""

    #...
    def __connect(self):
        """Open connection to remote host."""
        self.client = SSHClient()
        self.client.set_missing_host_key_policy(AutoAddPolicy())
        self.client.load_system_host_keys()
        self.client.connect(self.host,
                            username=self.user,
                            timeout=5000,
                            port=self.port,
                            passphrase=self.passphrase,
                            password=self.password
                            )
        self.scp = SCPClient(self.client.get_transport())
        return self.client

### Disconnecting

In [ ]:
class RemoteClient:
    #...

    def disconnect(self):
        """Close ssh connection."""
        self.client.close()
        self.scp.close()


### Executing Unix Commands

In [ ]:
class RemoteClient:
    #...

    def execute_commands(self, commands):
        """Execute multiple commands in succession."""
        if self.client is None:
            self.client = self.__connect()
        for cmd in commands:
            stdin, stdout, stderr = self.client.exec_command(cmd)
            stdout.channel.recv_exit_status()
            response = stdout.readlines()
            for line in response:
                logger.info(f'INPUT: {cmd} | OUTPUT: {line}')

    remote.execute_commands(['cd /var/www/ && ls',
                            'tail /var/log/nginx/access.log',
                            'ps aux | grep node'])

### Uploading (and Downloading) Files via SCP

    self.scp = SCPClient(self.client.get_transport())

In [ ]:
class RemoteClient:
    #...

    def bulk_upload(self, files):
        """Upload multiple files to a remote directory."""
        if self.client is None:
            self.client = self.__connect()
        uploads = [self.__upload_single_file(file) for file in files]
        logger.info(f'Finished uploading {len(uploads)} files to {self.remote_path} on {self.host}')

    def __upload_single_file(self, file):
        """Upload a single file to a remote directory."""
        try:
            self.scp.put(file,
                         recursive=True,
                         remote_path=self.remote_path)
        except SCPException as error:
            logger.error(error)
            raise error
        finally:
            logger.info(f'Uploaded {file} to {self.remote_path}')

In [ ]:
class RemoteClient:

    #...

    def download_file(self, file):
        """Download file from remote host."""
        if self.client is None:
            self.client = self.__connect()
        self.scp.get(file)

### Run the script

> npr.:  `export REMOTE_HOST=ssh-telnet-test`

In [ ]:
"""Perform tasks against a remote host."""
from client import RemoteClient
from files import fetch_local_files

host = 'ssh-telnet-test'
port = 22
# credentials
user='testuser'
password='test'
passphrase='plume'
# file upload
local_file_directory='/home/plume/vsebina/osnovni_tecaj/11_Interakcija_SSH_Telnet/skripte/ssh_scp_module/data'
remote_path='/home/testuser/data'

def remote = RemoteClient(host, user, passphrase=passphrase, password=password, remote_path=remote_path, port=22)():
    """Initialize remote host client and execute actions."""
    remote = RemoteClient(host, user, passphrase=passphrase, password=password, remote_path=remote_path, port=22)
    #execute_command_on_remote(remote)
    download_file_from_remote(remote)
    #upload_files_to_remote(remote)
    remote.disconnect()
    
def upload_files_to_remote(remote):
    """Upload files to remote via SCP."""
    files = fetch_local_files(local_file_directory)
    remote.bulk_upload(files)

def download_file_from_remote(remote):
    remote.download_file('/home/testuser/some.txt')


def execute_command_on_remote(remote):
    """Execute UNIX command on the remote host."""
    remote.execute_commands(['uname -a'])

main()

## Telnet

`class telnetlib.Telnet(host=None, port=0[, timeout])`

In [ ]:
from telnetlib import Telnet

with Telnet('localhost', 23) as tn:
    tn.interact()

`Telnet.read_until(expected, timeout=None)`

`Telnet.write(buffer)`

`Telnet.read_all()`

`getpass.getpass(prompt='Password: ', stream=None)`

[getpass — Portable password input](https://docs.python.org/3.8/library/getpass.html)

In [ ]:
import getpass
import telnetlib

HOST = "ssh-telnet-test"

user = 'testuser' 
#user = input("Enter your remote account: ")

password = 'test' 
#password = getpass.getpass()


with telnetlib.Telnet(HOST) as tn:
    tn.read_until(b"login: ")
    tn.write(user.encode('ascii') + b"\n")

    if password:
        tn.read_until(b"Password: ")
        tn.write(password.encode('ascii') + b"\n")

    tn.write(b"ls\n")
    tn.write(b"mkdir /home/testuser/telnet-test \n")

    tn.write(b"exit\n")

    print(tn.read_all().decode('ascii'))

### Primer uporabe: Telnet

In [ ]:
import getpass
import telnetlib

def create_dirs(dir_list, path, host, user, password, port=23):
    with telnetlib.Telnet(host, port) as tn:
        print(f"--> connected to host: {host} on port: {port}")
        
        tn.read_until(b"login: ")
        tn.write(user.encode('ascii') + b"\n")

        if password:
            tn.read_until(b"Password: ")
            tn.write(password.encode('ascii') + b"\n")
        
        print(f"--> succsessful login for user: {user}")
            
        for dir_name in dir_list:
            path_name = f'{path}/{dir_name}'
            command = f'mkdir -v {path_name}\n'
            tn.write(command.encode('ascii'))
            succsess_mesage = f"mkdir: created directory '{path_name}'".encode('ascii')
            res = tn.read_until(succsess_mesage, timeout=2)
            if succsess_mesage in res:
                print(f'--> {dir_name} created successfully')
            else:
                print(f'--> {dir_name} creation failed')

        tn.write(b"exit\n")
        print(tn.read_all().decode('ascii'))


create_dirs(['dir1', 'dir2', 'dir3'], path='/home/testuser/telnet-test', 
                    host='ssh-telnet-test',
                    user='testuser',
                    password='test')